In [1]:
## KAGGLE ONLY
from shutil import copyfile
copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

copyfile(src="../input/input2/train_unlabeld_dataloader.p", dst="../working/train_unlabelled_dataloader.p")
copyfile(src="../input/input2/train_labeled_dataloader.p", dst="../working/train_labelled_dataloader.p")

'../working/train_labelled_dataloader.p'

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *

from tqdm import tqdm_notebook as tqdm

# import evaluation
# import importlib
# importlib.reload(evaluation)

## Get Dataloaders

In [3]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [4]:
path = os.getcwd()
data_dir = path + '/'
# data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [5]:
train_loader_labelled = pkl.load(open(data_dir + 'train_labelled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabelled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [6]:
review_dict = pkl.load(open(data_dir + 'dictionary.p','rb'))

In [7]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [8]:
print(torch.__version__)

1.1.0


## PRE TRAINED WORD EMBEDDINGS 

In [9]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')

In [10]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [11]:
def build_matrix(review_dict, embedding_index ,dim = 200):
#     embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(review_dict.tokens), dim))
    unknown_words = []
    
    for word, i in review_dict.ids.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [12]:
glove_twitter = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt' #Change loc for local system
# glove_twitter = data_dir + 'glove.twitter.27B.200d.txt'

In [13]:
embedding_index = load_embeddings(glove_twitter)

In [14]:
glove_embedding_index,unknown_words = build_matrix(review_dict, embedding_index)
del embedding_index

In [15]:
len(review_dict.tokens)

16256

In [16]:
len(unknown_words)

4428

In [17]:
# for word in unknown_words:
#     print(word)

## Neural Network LSTM Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [18]:
class LSTM_model(nn.Module):
    """
    LSTM classification model using pretrained glove embeddings
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, embedding_matrix, num_hidden_layers = 3, hidden_size = 100, num_classes = 2):
        super(LSTM_model, self).__init__()
        vocab_size = embedding_matrix.shape[0]
        embed_size = embedding_matrix.shape[1]
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.num_hidden_layers = num_hidden_layers
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=0)    
        self.embed.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embed.weight.requires_grad = False
        
        self.lstm = nn.LSTM(self.embed_size,self.hidden_size,self.num_hidden_layers, batch_first=True,bidirectional= True,bias=True)
        
        self.projection = nn.Linear(2*self.hidden_size, self.num_classes, bias=True)

    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
#         print(embedding.shape) # below assumes "batch_size x num_tokens x Emb_dim" (VERIFY)
        
        lstm_output = self.lstm(embedding)
        # lstm_output is a tuple containing lstm output and (hidden_state, lstm_cell). 
        # lstm_output[0] would be of shape "batch_size x num_tokens x hidden_size" (VERIFY)
        
        logits = self.projection(lstm_output[0])
        # logits would be of shape "batch_size x num_tokens x num_classes (2)" (VERIFY)
        
        batch_size, _, __ = logits.shape
        
        #selecting the logit at the flagged index
        relevant_logits = logits[list(range(batch_size)),flagged_index]
        # relevant_logits would be of shape "batch_size x num_classes (2)" (VERIFY)
        
        return relevant_logits

## First performing fully supervised learning using the labelled set to train new vector representations

In [19]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

model = LSTM_model(glove_embedding_index, num_hidden_layers = 3, hidden_size = 100, num_classes = 2).to(current_device)

In [20]:
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

### Scratch work to understand the lstm output and the loss function

In [21]:
for i,X in enumerate(val_loader):
    tokens_labelled, labels, flagged_indices_labelled, = X
    break

In [22]:
tokens_labelled = tokens_labelled.to(current_device)
flagged_indices_labelled = flagged_indices_labelled.to(current_device)
labels = labels.to(current_device)

In [23]:
tokens_labelled.shape

torch.Size([32, 30])

In [24]:
embeddings = model.embed(tokens_labelled)
embeddings.shape

torch.Size([32, 30, 200])

In [25]:
lstm_output = model.lstm(embeddings)
lstm_output[0].shape

torch.Size([32, 30, 200])

In [26]:
logits = model.projection(lstm_output[0])
logits.shape

torch.Size([32, 30, 2])

In [27]:
relevant_logits = logits[list(range(32)),flagged_indices_labelled]
relevant_logits.shape

torch.Size([32, 2])

In [28]:
labels.shape

torch.Size([32])

In [29]:
loss = criterion(relevant_logits,labels)
loss

tensor(21.9358, grad_fn=<NllLossBackward>)

## Supervised model training

In [30]:
def train_supervised_model(model, criterion, optimizer, train_loader_labelled, valid_loader, num_epochs=10, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        total_epoch_loss = 0
        
        for i,(tokens_labelled, labels, flagged_indices_labelled) in enumerate(train_loader_labelled):
            
            tokens_labelled = tokens_labelled.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            labels = labels.to(current_device)

            # forward pass and compute loss
            logits = model(tokens_labelled,flagged_indices_labelled)
            
            loss = criterion(logits, labels)
        
            # run update step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += loss

            if i % print_every == 0:
                losses = loss/len(tokens_labelled)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader_labelled.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            logits = model(tokens,flagged_indices)
            
            loss = criterion(logits, labels)
            
            #Add loss to the validation loss
            total_validation_loss += loss

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"vocab_size":model.vocab_size, "embed_size": model.embed_size}
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
        
    return model, train_losses, val_losses

In [31]:
path = os.getcwd()
model_dir = path

In [32]:
len(train_loader_labelled)

27

In [33]:
lstm_model, train_losses, val_losses = train_supervised_model(model, criterion, optimizer, train_loader_labelled, val_loader, num_epochs=10, path_to_save=model_dir)


2019-11-22 21:04:50.251646 | Epoch 0
Average training loss at batch  0 : 0.679
Average training loss after epoch  0 : 0.560
Average validation loss after epoch  0 : 0.254
2019-11-22 21:04:55.801111 | Epoch 1
Average training loss at batch  0 : 0.302
Average training loss after epoch  1 : 0.307
Average validation loss after epoch  1 : 0.165
2019-11-22 21:05:01.663792 | Epoch 2
Average training loss at batch  0 : 0.212
Average training loss after epoch  2 : 0.224
Average validation loss after epoch  2 : 0.155
2019-11-22 21:05:07.570111 | Epoch 3
Average training loss at batch  0 : 0.109
Average training loss after epoch  3 : 0.166
Average validation loss after epoch  3 : 0.190
2019-11-22 21:05:13.440210 | Epoch 4
Average training loss at batch  0 : 0.103
Average training loss after epoch  4 : 0.151
Average validation loss after epoch  4 : 0.159
2019-11-22 21:05:19.358166 | Epoch 5
Average training loss at batch  0 : 0.060
Average training loss after epoch  5 : 0.094
Average validation lo

### Clustering Stuff (un-tailored)

In [36]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids, labelled = False,  cluster_assignments = None):
        if labelled:
            num_reviews = len(cluster_assignments)
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances = distances[list(range(num_reviews)),cluster_assignments]
            loss = self.lmbda * cluster_distances.sum()
        else:
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances, cluster_assignments = distances.min(1)
            loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [37]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)

## Dataloader stuff

In [38]:
def loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)
    except StopIteration:
        train_loader_labelled_iter = iter(train_loader_labelled)
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)

    return tokens, labels, flagged_indices, train_loader_labelled_iter


def loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)
    except StopIteration:
        train_loader_unlabelled_iter = iter(train_loader_unlabelled)
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)

    return tokens, labels, flagged_indices, train_loader_unlabelled_iter

### Training Function (un-tailored, needs alterations)

In [ ]:
def train_clusters(model, centroids, criterion, optimizer, train_loader_labelled, train_loader_unlabelled, valid_loader, num_epochs=10, num_batches = 1000, path_to_save=None, print_every = 1000):

    train_loader_labelled_iter = iter(train_loader_labelled)
    train_loader_unlabelled_iter = iter(train_loader_unlabelled)

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.eval()
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in range(num_batches):
            tokens_labelled, labels, flagged_indices_labelled, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            tokens_unlabelled, _, flagged_indices_unlabelled, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            tokens_labelled = tokens_labelled.to(current_device)
            labels = labels.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            
            tokens_unlabelled = tokens_unlabelled.to(current_device)
            flagged_indices_unlabelled = flagged_indices_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled = model(tokens_labelled,flagged_indices_labelled)
            sentence_embed_unlabelled = model(tokens_unlabelled,flagged_indices_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
    
            total_batch_loss = cluster_loss_labelled.data + cluster_loss_unlabelled.data
            
#             #Add loss to the epoch loss
            total_epoch_loss += total_batch_loss

#             # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments_labelled, sentence_embed_labelled)
    
            update_clusters(centroid_sums, centroid_counts,
                            cluster_assignments_unlabelled, sentence_embed_unlabelled)

            if i % print_every == 0:
                losses = total_batch_loss/(len(tokens_labelled)+ len(tokens_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"vocab_size":model.vocab_size, "embed_size": model.embed_size}
            torch.save(model.state_dict(), path_to_save+'model_dict.pt')
            torch.save(centroids, path_to_save+'centroids')
            torch.save(train_losses, path_to_save+'train_losses')
            torch.save(val_losses, path_to_save+'val_losses')
            torch.save(opts, path_to_save+'opts')
            
        
    return model, centroids, train_losses, val_losses

In [39]:
unsupervised_model = model
unsupervised_model.projection = nn.Identity()

In [40]:
# model = neuralNetBow(opts['vocab_size'], opts['emb_dim'])
centroids = centroid_init(2, 200,train_loader_labelled, unsupervised_model, current_device)
criterion = KMeansCriterion(1).to(current_device)
optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

In [41]:
centroids.shape

tensor([[ 4.6505e-01,  7.8163e-01,  4.8528e-01, -4.6817e-03,  1.9037e-02,
          1.6668e-01,  1.5806e-01,  2.1668e-04,  2.7631e-01,  1.3465e-01,
          8.0866e-01, -4.4393e-01,  3.4200e-02,  2.6709e-03, -1.8867e-02,
         -2.3933e-01, -2.6263e-01, -6.3401e-01, -2.9267e-01, -3.3880e-01,
          3.9647e-01,  1.1013e-01, -6.1875e-03, -2.3355e-03, -2.1256e-01,
         -8.5083e-01, -8.6541e-01,  2.5603e-01,  1.7084e-01,  2.1494e-02,
         -6.6351e-01, -3.8501e-02, -1.3520e-01, -5.3624e-02, -2.7725e-02,
         -1.6295e-02, -4.4198e-02, -9.1232e-02,  8.4665e-03,  1.6025e-01,
         -1.2624e-01, -3.1942e-01, -5.3474e-02, -4.1226e-01, -7.0297e-01,
         -1.9236e-01,  3.1778e-02,  4.0695e-01, -2.9812e-01, -4.6574e-02,
          2.8392e-01,  7.4745e-01, -3.1627e-01,  7.7724e-02,  2.9165e-01,
          2.3843e-01, -3.7677e-02,  3.6977e-01,  9.2153e-02,  4.5235e-01,
          7.1038e-01, -1.5485e-02,  2.5018e-01, -4.1925e-01,  1.2048e-03,
          4.1945e-01, -4.0571e-01,  6.

In [ ]:
path = os.getcwd()
model_folder= 'baseline_semisupervised_frozen_glove/'
model_dir = path + '/models/' + model_folder
model_dir = path

In [ ]:
num_batches = int(len(train_loader_unlabelled.dataset)/train_loader_unlabelled.batch_size)+1
num_batches

In [ ]:
lstm_model, lstm_centroids, lstm_train_losses, lstm_val_losses = train_clusters(unsupervised_model, centroids, criterion, optimizer, train_loader_labelled,train_loader_unlabelled, val_loader, num_epochs=10, num_batches=num_batches, path_to_save=model_dir)



In [ ]:
# #Only needed for Kaggle

# from IPython.display import FileLink, FileLinks 
# FileLinks('.') #lists all downloadable files on server